In [2]:
!pip install sentence-transformers
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.0 MB/s eta 0:00:00


In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Simulated knowledge base
knowledge_base = [
    "The capital of France is Paris.",
    "The Eiffel Tower is located in Paris.",
    "London is the capital of England.",
    "The United States has 50 states."
]

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Initialize retriever (using Sentence-BERT for embeddings)
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

# Precompute embeddings for knowledge base
kb_embeddings = retriever_model.encode(knowledge_base)

# Initialize generator (using T5 for this example)
tokenizer = AutoTokenizer.from_pretrained("t5-small")
generator = AutoModelForSeq2SeqLM.from_pretrained("t5-small")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
def retrieve(query, top_k=2):
    query_embedding = retriever_model.encode([query])[0]
    similarities = np.dot(kb_embeddings, query_embedding)
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [knowledge_base[i] for i in top_indices]

def generate(query, context):
    input_text = f"question: {query} context: {' '.join(context)}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = generator.generate(input_ids, max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def rag(query):
    retrieved_docs = retrieve(query)
    answer = generate(query, retrieved_docs)
    return answer



In [6]:
# Example usage
query = "What is the capital of France?"
result = rag(query)
print(f"Query: {query}")
print(f"Answer: {result}")

Query: What is the capital of France?
Answer: Paris
